#**Import the libraries**


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
from google.colab import files

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]

#**Create empty lists for holding the intermediate results. List A will hold Restaurant name, B will hold area name, C- city name, D-rating, E-Recommendations, F- # of Reviews, G-cost for two**

#**The scraper is going to store each web page's URL in a list url list. Then our scraper will visit each URL and load the source code of the page into right_table (this is known as web crawling) then from the source code, the scraper will scrape the data.**

In [ ]:
#url_list=['http://www.mouthshut.com/restaurants']
url_list=['https://www.mouthshut.com/Online-Shopping-ProID-925153']
for url in url_list:
    r=requests.get(url)
    data=r.text

    soup = BeautifulSoup(data, "lxml")
    links=soup.find_all('li',class_='page-item next')
    for link in links:
        cells = link.findAll('a',class_='page-link')
        url_list.append('http://www.mouthshut.com' +cells[0].get('href'))
    right_table=soup.find_all('div',class_='card-deck')
    for table in right_table:
        tbl1 = table.findAll('div',class_='listing-prod-card card list-view')
        for row in tbl1:
                cells = row.findAll('img',class_='image-object-fit-contain product-img b-lazy')
                string_ans=cells[0].get('title')
                A.append(string_ans.strip())
                tb12 = row.findAll('span',class_ = 'rating-no')
                if len(tb12)!=0:
                     B.append (tb12[0].find(text=True))
                tbl3 = row.findAll('span',class_='recommended-no')
                if len(tbl3)!=0:
                     C.append (tbl3[0].find(text=True))
                tbl4 = row.findAll('a',class_='review-no')
                if len(tbl4)!=0:
                     D.append (tbl4[0].find(text=True))
                tb15 = row.findAll('a',class_='product-img')
                reviewIds=tb15[0].get('href')
                E.append(reviewIds[reviewIds.rindex("-")+1:len(reviewIds)])

#**Paring the HTML source code using HTML tags and appending the data into the**

*   List item
*   List item

pre-defined lists

In [ ]:
df=pd.DataFrame(A, columns=['EcommerceSite'])
df['Rating']=B
df['Recommends']=C
df['Reviews']=D
df['ReviewId']=E         

In [ ]:
df.head(100)

,EcommerceSite,Rating,Recommends,Reviews,ReviewId
0,Kwabey.com,1,1%,(166 Votes),926052043
1,Tkameez.com,1,1%,(3 Votes),926033828
2,Ulykshop.in,0,0%,(0 Votes),926086405
3,Kaya Care Ayurveda,1,1%,(1 Vote),926049281
4,Peachmode.com,1.12,3%,"(2,045 Votes)",925738545
...,...,...,...,...,...
95,Tharakansdutypaid.com,0,0%,(0 Votes),926091308
96,Rustictown.com,0,0%,(0 Votes),926091095
97,Afcfurniture.in,0,0%,(0 Votes),926090220
98,Culttrendy.com,5,100%,(1 Vote),926090219


In [ ]:
def getReview(cname,cid,df):
      A1=[]
      B1=[]
      C1=[]
      D1=[]
      E1=[]
      F1=[]
      G1=[]
      for x in range(0, 80):
              if x == 1:
                  URL = "https://www.mouthshut.com/Review/rar_reviews.aspx?cname="+cname+"&cid="+cid
              else:
                  URL = "https://www.mouthshut.com/Review/rar_reviews.aspx?cname="+cname+"&cid="+cid+"&page={}".format(x)
                  #URL = "https://www.mouthshut.com/Review/rar_reviews.aspx?"+cname+"-reviews-"+cid+"-page-{}".format(x)
              r = requests.get(URL)
              soup = BeautifulSoup(r.content, 'html.parser')
              reviews = []  # a list to store reviews

              # i'm not going to use CSS selectors
              review_divs = soup.find_all('div', class_='row review-article')
              for review_div1 in review_divs :
                  review_div=review_div1.find('div', class_='col-10 review')
                  title_anchor = review_div.find('a')
                  if len(title_anchor)!=0:
                      title_text = title_anchor.find(text=True)
                      title_url = title_anchor['href']
                      rating_div = review_div.find('div', class_='rating')
                      rating_stars = len(rating_div.find_all('i', class_='icon-rating rated-star'))
                      views = rating_div.find('span', class_='total-views').find('span', class_='views').text
                      date = review_div.find('div', class_='rating').find_all('span')[1].text.strip()
                      review_text = review_div.find('div', class_='more reviewdata').text.strip()
                      review_text = review_text.replace('\n', ' ')
                      A1.append(title_text)
                      B1.append(title_url)
                      C1.append(rating_stars)
                      D1.append(date)
                      E1.append(views)
                      F1.append(review_text)
                      G1.append(cid)
      df_reviews=pd.DataFrame(A1, columns=['title_text'])
      df_reviews['title_url']=B1
      df_reviews['rating']=C1
      df_reviews['date']=D1
      df_reviews['views']=E1
      df_reviews['review_text']=F1
      df_reviews['reviewId']=G1
      df=pd.concat([df, df_reviews], axis=0)
      return df;

In [ ]:
reviews_df = pd.DataFrame(columns=['title_text','title_url','rating','date','views','review_text','reviewId'])
for ind in df.index:
     reviews_df=getReview(str(df['EcommerceSite'][ind]).replace('.','-'),df['ReviewId'][ind],reviews_df)
#reviews_df=getReview('Flipkart-com','925076148',reviews_df)
reviews_df.head(200)     

,title_text,title_url,rating,date,views,review_text,reviewId
0,Flipkart not provide service,https://www.mouthshut.com/review/Flipkart-com-...,3,10 hrs 58 mins ago,131 Views,Flipkart not delivered thinks in even smoll to...,925076148
1,Account has not working,https://www.mouthshut.com/review/Flipkart-com-...,3,1 day ago,149 Views,Dear sirMy account not working please re-activ...,925076148
2,Painted black rice,https://www.mouthshut.com/review/Flipkart-com-...,1,2 days ago,245 Views,POISON! When I opened the packet there was a s...,925076148
3,Its a Flopkart,https://www.mouthshut.com/review/Flipkart-com-...,1,2 days ago,220 Views,Mostly you are on luck when you order from Fli...,925076148
4,Flipkart failed to deliver my order,https://www.mouthshut.com/review/Flipkart-com-...,1,3 days ago,217 Views,Flipkart failed to deliver my order twice and ...,925076148
...,...,...,...,...,...,...,...
195,Froud from FLIPKART,https://www.mouthshut.com/review/Flipkart-com-...,1,"Mar 27, 2021 09:47 PM",742 Views,I have received voto brand mobile phone ..phon...,925076148
196,Worst experience in flipkart,https://www.mouthshut.com/review/Flipkart-com-...,1,"Mar 26, 2021 07:39 AM",685 Views,3weeks back I have ordered new laptop for my n...,925076148
197,Flipkart services have gone south,https://www.mouthshut.com/review/Flipkart-com-...,1,"Mar 25, 2021 07:41 PM",792 Views,Flipkart has been slipping on its services. Th...,925076148
198,Very poor service of flipkart Currier partner ...,https://www.mouthshut.com/review/Flipkart-com-...,1,"Mar 25, 2021 01:43 PM",682 Views,I bought a mobile phone POCO M3 which tracking...,925076148


In [ ]:
reviews_df.to_csv('df_reviews.csv', index = False)
df.to_csv('df_site_reviews.csv', index = False)
files.download("df_reviews.csv")
files.download("df_site_reviews.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>